In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import selenium.webdriver.support.ui as ui
from pdf2image import convert_from_path
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import requests
import random
import shutil
import glob
import json
import fire
import time
import csv
import sys
import os
import re

pytesseract.pytesseract.tesseract_cmd = r'E:\Tesseract-OCR\tesseract.exe'
base_dir = os.path.abspath(os.path.dirname("__file__"))
os.environ["LANG"] = "en_US.UTF-8"

# 192.41.71.199:3128   50.246.120.125:8080  157.230.44.213:8080   74.121.98.90:8080   192.41.71.221:3128 
# 74.121.98.90:8080  192.144.215.244:3128
PROXY =     "192.41.71.199:3128"   

options = webdriver.ChromeOptions()
appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}
options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
# options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
# options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"

driver = webdriver.Chrome(executable_path =r"C:\Users\Ankita Rajesh singh\Desktop\Chromedriver\chromedriver",options=options)
driver.maximize_window()
driver.get("https://www.amazon.com/s?k=sanitizer&crid=2QWKDX28H5D87&sprefix=sanitizer%2Caps%2C503&ref=nb_sb_ss_ts-a-p_1_9")


# GET SESSION ID & URL
executor_url = driver.command_executor._url
session_id = driver.session_id
print(session_id)
print(executor_url)


501f7ad2aa7c6ef213b0e562d2f03ba4
http://127.0.0.1:58654


In [4]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [5]:
results = soup.find_all('div', {'data-component-type':'s-search-result'})

In [6]:
len(results)

60

# Prototype the record

In [7]:
item = results[0]

In [8]:
atag = item.h2.a

In [9]:
description = atag.text.strip()

In [10]:
url ='https://www.amazon.com/' + atag.get('href')

In [11]:
url

'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A1013952NLBANBC8NJRG&url=%2FSanitizer-Sterilizer-Smartphone-Clinically-59S%2Fdp%2FB0861BNBXW%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2QWKDX28H5D87%26dchild%3D1%26keywords%3Dsanitizer%26qid%3D1607008349%26sprefix%3Dsanitizer%252Caps%252C503%26sr%3D8-1-spons%26psc%3D1&qualifier=1607008349&id=2339765625581557&widgetName=sp_atf'

In [12]:
price_parent = item.find('span','a-price')

In [13]:
price = price_parent.find('span','a-offscreen').text

In [14]:
price

'$79.99'

In [15]:
prodct_price = price.replace('$','')
prodct_price

'79.99'

In [16]:
rating = item.i.text
rating

'4.6 out of 5 stars'

In [17]:
review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
review_count

'631'

In [44]:
img_url = item.find('span',{'class':"a-button a-button-thumbnail a-button-toggle"})

In [46]:
img_url

# Generalize the pattern

In [22]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url ='https://www.amazon.com/' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text 
        prodct_price = price.replace('$','')
    except AttributeError:
        return
    
    try:
        #rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
    result = (description, price, rating, review_count, url)
    return result

In [ ]:
img_url  =

In [24]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [25]:
records[0]

('Automatic Hand Sanitizer Dispenser, 1200ml Hand Sanitizer Stand - Touchless Auto Sensor Sanitizing Station - Ship From USA- Steel Floor Stand, Drip Catcher - Ideal for All Public Places',
 '$138.98',
 '4.7 out of 5 stars',
 '106',
 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09848053A6J15VHQOKEJ&url=%2FAutomatic-Sanitizer-Dispenser-1200ml-Stand%2Fdp%2FB08GKVMJG5%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2QWKDX28H5D87%26dchild%3D1%26keywords%3Dsanitizer%26qid%3D1606923473%26sprefix%3Dsanitizer%252Caps%252C503%26sr%3D8-1-spons%26psc%3D1&qualifier=1606923473&id=8609005465985877&widgetName=sp_atf')

In [26]:
for row in records:
    print(row[1])

$138.98
$138.98
$79.99
$49.99
$39.99
$119.99
$15.99
$78.99
$109.99
$199.99
$12.00
$138.98
$139.00
$69.99
$59.99
$14.99
$34.99
$55.99
$49.99
$23.99
$34.99
$69.99
$27.99
$55.99
$115.99
$27.99
$51.99
$85.99
$15.95
$65.99
$18.91
$39.99
$25.73
$57.98
$16.28
$35.99
$69.99
$59.99
$69.88
$69.99
$129.95
$59.99
$129.99
$98.95
$59.99


# Getting the next page

In [27]:
def get_url(search_term):
    template ='https://www.amazon.com/s?k=sanitizer&crid=2QWKDX28H5D87&sprefix=sanitizer%2Caps%2C503&ref=nb_sb_ss_ts-a-p_1_9'
    search_term = search_term.replace('','+')
    
    #add query to url
    url = template.format(search_term)
    
    #add page query
    url += '&page{}'
    
    return url 

# Putting it all together

In [32]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import selenium.webdriver.support.ui as ui
from pdf2image import convert_from_path
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import requests
import random
import shutil
import glob
import json
import fire
import time
import csv
import sys
import os
import re

def get_url(search_term):
    template ='https://www.amazon.com/s?k=sanitizer&crid=2QWKDX28H5D87&sprefix=sanitizer%2Caps%2C503&ref=nb_sb_ss_ts-a-p_1_9'
    search_term = search_term.replace('','+')
    
    #add query to url
    url = template.format(search_term)
    
    #add page query
    url += '&page{}'
    
    return url 


def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    url ='https://www.amazon.com/' + atag.get('href')
    
    try:
        #price
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text 
        prodct_price = price.replace('$','')
    except AttributeError:
        return
    
    try:
        #rating
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
    result = (description, price, rating, review_count, url)
    return result




def main(search_term):
    options = webdriver.ChromeOptions()
    options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
    options.headless = False
    
    record = []
    url = get_url(search_term)
    
    for page in range(1,8):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source,'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    with open('results.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','url'])
        writer.writerows(records)

In [34]:
main('sanitize')